In [2]:
#Create Folder for modelling checkpoint
import os
import pandas as pd
# pickle to save dictionary in file
import pickle 
cwd = os.path.normpath(os.getcwd()+ os.sep + os.pardir + os.sep + os.pardir)

In [3]:
def loadResultDictionary(cwd, dictinaryName):
    with open(cwd + f'/results/{dictinaryName}.pkl', 'rb') as f:
        dictionary = pickle.load(f)
    return dictionary

def getMetricsForClientInCluster(dictionary, model, horizon, feature, cluster, client, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Federated'][model][horizon][feature][cluster][client]['MSE']
    rmse = rmse+ dictionary['Federated'][model][horizon][feature][cluster][client]['RMSE']
    mape = mape + dictionary['Federated'][model][horizon][feature][cluster][client]['MAPE']
    mae = mae + dictionary['Federated'][model][horizon][feature][cluster][client]['MAE']
    time = time + dictionary['Federated'][model][horizon][feature][cluster][client]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForClientInLocal(dictionary, model, horizon, feature, client, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Local'][model][horizon][feature][client]['MSE']
    rmse = rmse+ dictionary['Local'][model][horizon][feature][client]['RMSE']
    mape = mape + dictionary['Local'][model][horizon][feature][client]['MAPE']
    mae = mae + dictionary['Local'][model][horizon][feature][client]['MAE']
    time = time + dictionary['Local'][model][horizon][feature][client]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForCentral(dictionary, model, horizon, feature, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Central'][model][horizon][feature]['MSE']
    rmse = rmse+ dictionary['Central'][model][horizon][feature]['RMSE']
    mape = mape + dictionary['Central'][model][horizon][feature]['MAPE']
    mae = mae + dictionary['Central'][model][horizon][feature]['MAE']
    time = time + dictionary['Central'][model][horizon][feature]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForCluster(cluster_length, mse, rmse, mape, mae, time):
    mse = mse/cluster_length
    rmse = rmse /cluster_length
    mape = mape/cluster_length
    mae = mae/cluster_length
    time = time/cluster_length
    return mse, rmse, mape, mae, time

def namestr(obj, namespace):
    """
    Get the name of the variable as a String
    """
    return [name for name in namespace if namespace[name] is obj][0]

def getResultsFromFederatedDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Federated']:
            
            #initilize cluster_length and metrics
            mse, rmse, mape, mae, time = 0,0,0,0,0
            #For each of the clusters 0-5
            for cluster in dictionary['Federated'][model][horizon][feature]:
                #print("Model: ", model, "h: ", horizon, "f: ", feature)
                
                #Get the metrics for each client and add them up
                for idx, client in enumerate(dictionary['Federated'][model][horizon][feature][cluster]):
                    mse, rmse, mape, mae, time = getMetricsForClientInCluster(
                        dictionary, model, horizon, feature,
                        cluster, client, mse, rmse, mape, mae, time
                    )
                    
            #Devide the metrics by the number of clients in cluster to get average
            mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Federated', horizon, feature, model, mse, rmse, mape, mae, time]
            
    return result_df

def getResultsFromCentralDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Central']:

            mse, rmse, mape, mae, time = 0,0,0,0,0

            #Get the metrics for each client and add them up
            for idx, client in enumerate(dictionary['Central'][model][horizon][feature]):
                mse, rmse, mape, mae, time = getMetricsForCentral(
                    dictionary, model, horizon, feature,
                    mse, rmse, mape, mae, time
                )
           
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Central', horizon, feature, model, mse, rmse, mape, mae, time]

    return result_df


def getResultsFromLocalDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Local']:

            #initilize cluster_length and metrics
            mse, rmse, mape, mae, time = 0,0,0,0,0

            #Get the metrics for each client and add them up
            for idx, client in enumerate(dictionary['Local'][model][horizon][feature]):

                mse, rmse, mape, mae, time = getMetricsForClientInLocal(
                    dictionary, model, horizon, feature,
                    client, mse, rmse, mape, mae, time
                )
                
            #Devide the metrics by the number of clients in cluster to get average
            mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
            #Add metrics to the last row of the result_df
            result_df.loc[len(result_df)]= ['Local', horizon, feature, model, mse, rmse, mape, mae, time]

    return result_df

In [28]:
fl_H12_F5 = loadResultDictionary(cwd, 'Federated_results_H12_F5_3month')

result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])  


horizon = 'H12'
feature = 'F5'

#For LSTM, CNN, Transformer
for model in fl_H12_F5['Federated']:

    #initilize metrics
    mse, rmse, mape, mae, time = 0,0,0,0,0
    #For each of the clusters 0-5
    for cluster in fl_H12_F5['Federated'][model][horizon][feature]:

        #Get the metrics for each client and add them up
        for idx, client in enumerate(fl_H12_F5['Federated'][model][horizon][feature][cluster]):
            mse, rmse, mape, mae, time = getMetricsForClientInCluster(
                fl_H12_F5, model, horizon, feature,
                cluster, client, mse, rmse, mape, mae, time
            )

        #Devide the metrics by the number of clients in cluster to get average
    mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
    #print(cluster, " : ",mse, rmse, mape, mae, time)
    result_df.loc[len(result_df)]= ['Federated', horizon, feature, model, mse, rmse, mape, mae, time]

df_f_results_f = result_df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_f_results_f

C:\Users\rs1044\AppData\Local\Temp\ipykernel_24636\2810705840.py:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results_f = result_df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.053160  0.224982  7.879764e+07  0.184078   
                LSTM         0.042593  0.200439  7.315450e+07  0.159479   
                Transformer  0.041271  0.196789  7.092372e+07  0.153641   

                                 Time  
Horizon Feature Model                  
H12     F5      CNN          0.029086  
                LSTM         0.476289  
                Transformer  0.164466

In [12]:
df_f_results_l

MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.050690  0.212491  1.188155e+08  0.184440   
                LSTM         0.041074  0.193626  9.432320e+07  0.168415   
                Transformer  0.035286  0.178702  8.120942e+07  0.149742   

                                  Time  
Horizon Feature Model                   
H12     F5      CNN           2.561662  
                LSTM         44.219390  
                Transformer  20.455862

In [11]:
#Local H12_F7
y=[2, 2, 4, 4, 1, 5, 1, 5, 1, 5, 5, 5, 5, 1, 3, 0, 3, 0, 5, 1, 3, 3, 3, 3, 5, 2, 2, 1, 3, 0, 0, 1, 2]
df = pd.read_csv(cwd+'/data/d03_data_processed/d03_data_processed_3month.csv', encoding= 'unicode_escape', index_col='Date')
#Display smart meter names and amount
smart_meter_names = df.columns[2:-4]

l_H12_F5 = loadResultDictionary(cwd, 'Local_results_H12_F5_3month_1y')

#Add cluster to the client
for idx, client in enumerate(smart_meter_names):
    # Füge die clienten ein, die zu dem Cluster gehören
    l_H12_F5['Local']['LSTM']['H12']['F5'][client]['Cluster'] = y[idx]
    l_H12_F5['Local']['CNN']['H12']['F5'][client]['Cluster'] = y[idx]
    l_H12_F5['Local']['Transformer']['H12']['F5'][client]['Cluster'] = y[idx]
    
result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])

horizon = 'H12'
feature = 'F5'

# For each of the models LSTM, CNN, Transformer
for model in l_H12_F5['Local']:

    #initilize cluster_length and metrics
    mse, rmse, mape, mae, time = 0,0,0,0,0

    #Get the metrics for each client and add them up
    for idx, client in enumerate(l_H12_F5['Local'][model][horizon][feature]):
        mse = mse + l_H12_F5['Local'][model][horizon][feature][client]['MSE']
        rmse = rmse + l_H12_F5['Local'][model][horizon][feature][client]['RMSE']
        mape = mape + l_H12_F5['Local'][model][horizon][feature][client]['MAPE']
        mae = mae + l_H12_F5['Local'][model][horizon][feature][client]['MAE']
        time = time + l_H12_F5['Local'][model][horizon][feature][client]['Time']
        #cluster = l_H12_F5['Local'][model][horizon][feature][client]['Cluster']
        #Add metrics to the last row of the result_df
    mse, rmse, mape, mae, time = getMetricsForCluster(33, mse, rmse, mape, mae, time)
    result_df.loc[len(result_df)]= ['Local', horizon, feature, model, mse, rmse, mape, mae, time]
    
df_f_results_l = result_df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_f_results_l


C:\Users\rs1044\AppData\Local\Temp\ipykernel_24636\1983287613.py:39: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results_l = result_df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.050690  0.212491  1.188155e+08  0.184440   
                LSTM         0.041074  0.193626  9.432320e+07  0.168415   
                Transformer  0.035286  0.178702  8.120942e+07  0.149742   

                                  Time  
Horizon Feature Model                   
H12     F5      CNN           2.561662  
                LSTM         44.219390  
                Transformer  20.455862

In [5]:
#Load all 4 federated results dictionaries
#fl_H12_F7 = loadResultDictionary(cwd, 'Federated_results_H12_F7')
#fl_H24_F7 = loadResultDictionary(cwd, 'Federated_results_H24_F7')
fl_H12_F5 = loadResultDictionary(cwd, 'Federated_results_H12_F5_3month_1y')
fl_H24_F5 = loadResultDictionary(cwd, 'Federated_results_H24_F5_3month_1y')

#fl_H12_F7, fl_H24_F7,
df_f = getResultsFromFederatedDictionary([ fl_H12_F5, fl_H24_F5])

df_f_results = df_f.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_f_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_21060\744069557.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_f_results = df_f.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.049524  0.210484  1.156910e+08  0.181810   
                LSTM         0.035963  0.180855  8.570923e+07  0.154001   
                Transformer  0.034521  0.176897  8.074714e+07  0.147860   
H24     F5      CNN          0.046985  0.205917  1.106717e+08  0.178310   
                LSTM         0.040595  0.192522  9.666221e+07  0.168193   
                Transformer  0.034773  0.177560  8.300803e+07  0.149873   

                                  Time  
Horizon Feature Model                   
H12     F5      CNN           3.741763  
                LSTM         46.833978  
                Transformer  24.173518  
H24     F5      CNN           4.130177  
                LSTM         54.441109  
                Transformer  28.475217

In [9]:
#Load all 4 federated results dictionaries
c_H12_F7 = loadResultDictionary(cwd, 'Central_results_H12_F7')
c_H24_F7 = loadResultDictionary(cwd, 'Central_results_H24_F7')
c_H12_F5 = loadResultDictionary(cwd, 'Central_results_H12_F5')
c_H24_F5 = loadResultDictionary(cwd, 'Central_results_H24_F5')

df_c = getResultsFromCentralDictionary([c_H12_F7, c_H24_F7, c_H12_F5, c_H24_F5])
df_c_results = df_c.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_c_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_17632\1490845994.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_c_results = df_c.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.488511  1.562867  1.099057e+09  1.401436   
                LSTM         0.470900  1.534437  1.140369e+09  1.405556   
                Transformer  0.206473  1.016054  6.208793e+08  0.888174   
        F7      CNN          0.657494  1.813138  1.341811e+09  1.626052   
                LSTM         0.470904  1.534443  1.140370e+09  1.405557   
                Transformer  0.201017  1.002538  6.111885e+08  0.884497   
H24     F5      CNN          0.456040  1.510034  1.084901e+09  1.368384   
                LSTM         0.472595  1.537200  1.142905e+09  1.407908   
                Transformer  0.216681  1.040867  6.495527e+08  0.933119   
        F7      CNN          0.415402  1.441183  1.016125e+09  1.311138   
                LSTM         0.472597  1.537205  1.142907e+09  1.407910   
                Transformer  0.208946  1.022120  6.163708e+08  0.906014   

                                   Time  
Horizon Feature Model                    
H12     F5      CNN           13.318900  
                LSTM         508.136090  
                Transformer  385.457190  
        F7      CNN           51.337567  
                LSTM         750.632808  
                Transformer  320.880664  
H24     F5      CNN           26.636164  
                LSTM         495.053867  
                Transformer  513.301120  
        F7      CNN           32.737389  
                LSTM         793.414520  
                Transformer  213.256256

In [6]:
#Load all 4 federated results dictionaries
#l_H12_F7 = loadResultDictionary(cwd, 'Local_results_H12_F7')
#l_H24_F7 = loadResultDictionary(cwd, 'Local_results_H24_F7')
l_H12_F5 = loadResultDictionary(cwd, 'Local_results_H12_F5_3month_1y')
l_H24_F5 = loadResultDictionary(cwd, 'Local_results_H24_F5_3month_1y')
#l_H12_F7, l_H24_F7
df_l = getResultsFromLocalDictionary([l_H12_F5, l_H24_F5, ])
df_l_results = df_l.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df_l_results

C:\Users\jonas\AppData\Local\Temp\ipykernel_21060\1385463017.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_l_results = df_l.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.050690  0.212491  1.188155e+08  0.184440   
                LSTM         0.041074  0.193626  9.432320e+07  0.168415   
                Transformer  0.035286  0.178702  8.120942e+07  0.149742   
H24     F5      CNN          0.047166  0.206544  1.127164e+08  0.179625   
                LSTM         0.041690  0.195026  9.590579e+07  0.170120   
                Transformer  0.037376  0.184444  8.730299e+07  0.158323   

                                  Time  
Horizon Feature Model                   
H12     F5      CNN           2.561662  
                LSTM         44.219390  
                Transformer  20.455862  
H24     F5      CNN           3.046750  
                LSTM         54.789618  
                Transformer  25.760575